In [3]:
#import modules
from lxml import html
import re
import requests
import pandas as pd
from datetime import datetime

<h2>Question 1<h2>

In [4]:
#get html and tree
html_page_link = 'candidateEvalData/webpage.html'
html_tree = html.parse(html_page_link)

data = {}

# parse artist name
artistName = html_tree.xpath("//h1[@class='lotName']//text()")[0]
data["Artist Name"] = artistName

#parse painting name
paintingName = html_tree.xpath("//h2[@class='itemName']//text()")[0]
data["Painting Name"] = paintingName

#parse price GBP
realisedPriceGBP = html_tree.xpath("//div[@class='price']//span[contains(@id,'PriceRealizedPrimary')]//text()")[0].strip("GBP")
data["Realised Price GBP"] =int(realisedPriceGBP.replace(",", ""))

#parse price US
realisedPriceUSD = html_tree.xpath("//div[contains(@id,'PriceRealizedSecondary')]//text()")[0].strip("USD")
data["Realised Price USD"] = int(realisedPriceUSD.replace(",", ""))

#parse price GBP est
estimatePriceGBP = html_tree.xpath("//div[@class='price']//span[contains(@id,'PriceEstimatedPrimary')]//text()")[0].strip("GBP")
data["Estimated Price GBP"] = int(realisedPriceGBP.replace(",", ""))

#parse price US est
estimatePriceGBP = html_tree.xpath("//span[contains(@id,'PriceEstimatedSecondary')]//text()")[0].strip("USD")
data["Estimated Price USD"] = int(realisedPriceUSD.replace(",", ""))

#image link
imageLink = html_tree.xpath("//div[@id='MainContentImage']//img[@id='imgLotImage']//@src")[0]
data["Image Link"] = imageLink

df = pd.DataFrame(data, index=['1'])
df.head()

,Artist Name,Painting Name,Realised Price GBP,Realised Price USD,Estimated Price GBP,Estimated Price USD,Image Link
1,Peter Doig (b. 1959),The Architect's Home in the Ravine,11282500,16370908,11282500,16370908,http://www.christies.com/lotfinderimages/D5973...


<h2>Question 2<h2>

In [5]:
dim_df = pd.read_csv("candidateEvalData/dim_df_correct.csv")
dim_df

,rawDim,height,width,depth
0,19×52cm,19.0,52.0,NaN
1,"50 x 66,4 cm",50.0,66.4,NaN
2,168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.),168.9,274.3,3.8
3,Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Im...,35.6,25.1,NaN
4,5 by 5in,12.7,12.7,NaN


In [6]:


# 19 x 52cm
reg1 = r"(?P<height>\d+)×(?P<width>\d+)(?P<unit>cm)"
re_search1 = re.search(reg1, dim_df["rawDim"][0])
print(float(re_search1.group("height")), float(re_search1.group("width")), re_search1.group("unit"))

# 50 x 66,4 cm
reg2 = r"(?P<height>\d+\,?\d+)\s?x\s?(?P<width>\d+\,\d+)\s?(?P<unit>cm)"
re_search2 = re.search(reg2, dim_df["rawDim"][1])
width = re_search2.group("width").replace(",","")
print(float(re_search2.group("height")), float(width), re_search2.group("unit"))

# 168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.)
reg3 = r"(?P<height>\d+\.?\d+)?(\s?x\s?)(?P<width>\d+\.?\d+)?(\s?x\s?)?(?P<depth>\d+\.?\d+)?\s?(?P<unit>cm)"
re_search3 = re.search(reg3, dim_df["rawDim"][2])
print(float(re_search3.group("height")), float(re_search3.group("width")), float(re_search3.group("depth")), re_search3.group("unit"))

# Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Image: 14 × 9 7/8 in. (35.6 × 25.1 cm)
reg4 = r"Image.*\((?P<height>\d+\.?\d+)?(\s?×\s?)(?P<width>\d+\.?\d+)?(\s?×\s?)?(?P<depth>\d+\.?\d+)?\s?(?P<unit>cm)"
re_search4 = re.search(reg4, dim_df["rawDim"][3])
print(float(re_search4.group("height")), float(re_search4.group("width")), re_search4.group("unit"))

# 5 by 5in
inch_to_cm = 2.54
reg5 = r"(?P<height>\d+)\s?by\s?(?P<width>\d+)\s?(?P<unit>in)"
re_search5 = re.search(reg5, dim_df["rawDim"][4])
print(float(re_search5.group("height")) * inch_to_cm, float(re_search5.group("width")) * inch_to_cm, re_search5.group("unit"))

19.0 52.0 cm
50.0 664.0 cm
168.9 274.3 3.8 cm
35.6 25.1 cm
12.7 12.7 in


<h2>Now combing the regex into one function<h2>

In [7]:
inch_to_cm = 2.54

# All of these variables could be expanded e.g. more synonyms such as inches
units = r"cm|in" 

by_seperators = r"by|x|×"

unit_seperators = r"[,\.]"

# At the moment only gets decimal numbers not fractions e.g. won't match 16 1/4 x 12 1/4 inch
decimal_number = fr"\d+{unit_seperators}?\d*"

dimensions_regex = (fr"(?P<height>{decimal_number})\s{{0,4}}(?:{by_seperators})?\s{{0,4}}"
                    fr"(?P<width>{decimal_number})\s{{0,4}}(?:{by_seperators})?\s{{0,4}}"
                    fr"(?P<depth>{decimal_number})?\s{{0,4}}(?P<unit>{units})")

# If this word appears after a dimension then discard it.
negative_lookahead = "image"

def formatInput(value, unit):
    # Remove "," from string
    # Convert from cm to inches
    value = float(value.replace(",","."))
    if unit == "in":
        value *= inch_to_cm
    return value


data = []
for dim in dim_df["rawDim"]:
    data_sub_row = []
    data_sub_row.append(dim)
    re_searchs = re.finditer(dimensions_regex, dim, re.IGNORECASE)
    for re_search in re_searchs:
        if re.search(negative_lookahead, dim[re_search.end():], re.IGNORECASE):
            continue
        for key in ("height", "width", "depth"):
            if not re_search.group(key):
                continue
            unit = re_search.group("unit")
            dimension_value = formatInput(re_search.group(key), unit)
            data_sub_row.append(dimension_value)
        break
    data.append(data_sub_row)

df = pd.DataFrame(data, columns=["rawDim", "height", "width", "depth"])
df

,rawDim,height,width,depth
0,19×52cm,19.0,52.0,NaN
1,"50 x 66,4 cm",50.0,66.4,NaN
2,168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.),168.9,274.3,3.8
3,Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Im...,35.6,25.1,NaN
4,5 by 5in,12.7,12.7,NaN


<h2>Question3<h3>
<h3>Please `scrapy crawl bearspace -O BearSpacePaintings.csv` from inside of the HeniWebCrawler directory before running this next cell<h1>

In [17]:
bear_space_paintings_df = pd.read_csv("HeniWebCrawler/BearSpacePaintings.csv")
bear_space_paintings_df.head()

,url,title,media,height_cm,width_cm,price_gbp
0,https://www.bearspace.co.uk/product-page/el-dorado-by-john-greenwood,El Dorado by John Greenwood,Oil on Gessoed canvas on Board,45.0,38.0,"3,800.00"
1,https://www.bearspace.co.uk/product-page/conversation-2-by-vic-wright,Conversation 2 by Vic Wright,Black and white casting cement with 24k gold leaf,12.0,8.0,320.00
2,https://www.bearspace.co.uk/product-page/constructed-veneer-vii-by-olly-fathers,Constructed Veneer VII by Olly Fathers,"Various wood veneers, various woods, board",34.0,25.0,940.00
3,https://www.bearspace.co.uk/product-page/disperse-by-dionne-cole,Disperse by Dionne Cole,76 x 101.6 cm,76.0,101.6,"1,400.00"
4,https://www.bearspace.co.uk/product-page/conversation-by-vic-wright,Conversation by Vic Wright,18K gold leaf and white casting cement,16.5,13.0,440.00


<h2>Question 4<h3>

<h3>Task 1 : Describe inner join, left join, right join, full join.<h3>

Inner Join = Selects all the rows from the left and right table where there is a matching value in the field you are joining on.
Left Join = Selects all of the rows from the left table and only the rows that match the join condition from the right table. 
Right Join = Selects all of the rows from the right table and only the rows that match the join condition from the left.
Full Join = Selects all of the rows from the left table and all of the rows from the right table. 

In [8]:
flights = pd.read_csv("candidateEvalData/flights.csv")
airports = pd.read_csv("candidateEvalData/airports.csv")
weather = pd.read_csv("candidateEvalData/weather.csv")
airlines = pd.read_csv("candidateEvalData/airlines.csv")

# Add full airline name to the flights dataframe and show the arr_time, origin, dest and the name of the airline.
SELECT f.arr_time, f.origin, f.dest, a.name FROM flights as f
INNER JOIN airlines as a
ON f.carrier == a.carrier;

# Filter resulting data.frame to include only flights containing the word JetBlue
SELECT f.arr_time, f.origin, f.dest, a.name FROM flights as f
INNER JOIN airlines as a
ON f.carrier == a.carrier
WHERE a.name LIKE "%JetBlue%";

# Filter resulting data.frame to include only flights containing the word JetBlue
SELECT f.arr_time, f.origin, f.dest, a.name FROM flights as f
INNER JOIN airlines as a
ON f.carrier == a.carrier
WHERE a.name LIKE "%JetBlue%";

In [4]:
# Add full airline name to the flights dataframe and show the arr_time, origin, dest and the name of the airline.
flights_fullname_df = pd.merge(flights, airlines, on="carrier", how="left")
task_2_1 = flights_fullname_df[["arr_time", "origin", "dest", "name"]]
task_2_1.head()

,arr_time,origin,dest,name
0,830.0,EWR,IAH,United Air Lines Inc.
1,850.0,LGA,IAH,United Air Lines Inc.
2,923.0,JFK,MIA,American Airlines Inc.
3,1004.0,JFK,BQN,JetBlue Airways
4,812.0,LGA,ATL,Delta Air Lines Inc.


In [5]:
task_2_2 = task_2_1[task_2_1["name"].str.contains("JetBlue")]
task_2_2.head()

,arr_time,origin,dest,name
3,1004.0,JFK,BQN,JetBlue Airways
6,913.0,EWR,FLL,JetBlue Airways
8,838.0,JFK,MCO,JetBlue Airways
10,849.0,JFK,PBI,JetBlue Airways
11,853.0,JFK,TPA,JetBlue Airways


In [12]:
task_2_3 = task_2_2.groupby(["origin"]).size().reset_index(name='counts').sort_values(by=["counts"], ascending=True)
task_2_3 = task_2_3.rename(columns={"counts" : "numFlights"})
task_2_3.head()

,origin,numFlights
2,LGA,21
0,EWR,24
1,JFK,148


In [13]:
task_2_4 = task_2_3[task_2_3["numFlights"] > 100]
task_2_4.head()

,origin,numFlights
1,JFK,148


<h2>What could have been better if I had more time<h3>

1) Improve the regex to extract more dimensions imformation

There are some paintings that have been scraped that are missing height and width information even though it's on the page. For example this listing, my spider doesn't extract the height and width but it could do if the dimensions regex was improved. In this case it'd need to handle the W and H characthers in the dimenions.
e.g. https://www.bearspace.co.uk/product-page/shades-of-coral-ii-by-dionne-cole	

2) Use item loaders for the scrapy spider.

At the moment the data is being inputted into a dictionary, but a better way would be to use scrapy itemloaders since you have the option to define the set fields that you want to scrape and also you can add input processors. For example you could strip the text inputs to remove any trailing white spaces and perform any other string cleaning that is needed. 

3) Use a linter

At the moment there is no linter being used for this project, it'd be good to add a sytnax checker and a code formatter so that all the code is formatted the same and follows a particular guidline. 